In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [3]:
class DSO:
	def __init__(self, totalHours, marginalCost, unitSalePrice):
		self.totalHours = totalHours
		self.numberOfAggregators = 0
		self.marginalCost = marginalCost
		self.unitSalePrice = unitSalePrice
		self.aggregators = []
		self.maxUtility = 0

	def addAggregator(self, aggregator):
		self.aggregators.append(aggregator)
		self.numberOfAggregators += 1

	def getAggregators(self):
		return self.aggregators

	def Sfunction(self, aggregator, tIndex):
		return aggregator.maximumDemand[tIndex] * self.maxPriceReference * (1 - np.exp(-aggregator.preferenceSatisfaction[tIndex] * (aggregator.actualLoad[tIndex] / aggregator.nominalDemand[tIndex])))
	
	def CostFunction(self, aggregator, tIndex):
		# return self.marginalCost[tIndex] * aggregator.actualLoad[tIndex]
		return 

	def utilityFunction(self, m, theta, omega):
		sum = 0
		for i in range(self.numberOfAggregators):
			agg = self.aggregators[i]
			for j in range(self.totalHours):
				sum += self.unitSalePrice[j] * agg.actualLoad[j] - self.CostFunction[j] * agg.actualLoad[j] + omega * self.Sfunction(agg, j)
		sum -= theta*m
		return sum
	
	def optimalPrice(self):
		return self.utilityFunction(0, 0, 0)

class Aggregator:
	def __init__(self, totalHours, nominalDemand, minimumDemand, maximumDemand, maxPriceReference, preferenceSatisfaction, actualLoad):
		self.totalHours = totalHours
		self.nominalDemand = nominalDemand
		self.minimumDemand = minimumDemand
		self.maximumDemand = maximumDemand
		self.maxPriceReference = maxPriceReference
		self.preferenceSatisfaction = preferenceSatisfaction
		self.actualLoad = actualLoad
		self.currentUtility = 0
		self.bestLoadResponse = []

	def utilityFunction(self, DSO):
		sum = 0
		for i in range(DSO.totalHours):
			sum += DSO.SFunction(self, i) - DSO.unitSalePrice[i] * self.actualLoad[i]
		return sum

	def optimalDR(self, DSO):
		for j in range(DSO.totalHours):
			self.bestLoadResponse[j] = (DSO.nominalDemand[j] / self.preferenceSatisfaction[j]) * (np.log((self.preferenceSatisfaction[j] * self.maxPriceReference) / DSO.unitSalePrice[j]))


In [ ]:
datasetPath = '../dataset/'

In [4]:
numOfLA = 3
numOfHours = 25919
dso = DSO()
LAs = []

# for i in range(3):
#     # read data from csv file
# 	letter = chr(ord('A') + i)
# 	fileName = 'Home' + letter + '_hourly.csv'
# 	df = pd.read_csv(datasetPath + fileName)
    

In [ ]:
# DSO gives the price for each hour
for i in range(numOfLA):
	la = LAs[i]
	la.optimalDR(dso)

In [ ]:
# DSO takes back the load response from each aggregator
dso.optimalPrice(LAs)

In [ ]:
# import gameanalysis
# import numpy as np

# # Define the DSO and aggregator objects
# totalHours = 24
# marginalCost = np.random.rand(totalHours)
# unitSalePrice = np.random.rand(totalHours)
# maxPriceReference = 1
# preferenceSatisfaction = np.random.rand(totalHours)
# actualLoad = np.random.rand(totalHours)
# nominalDemand = np.random.rand(totalHours)
# minimumDemand = np.random.rand(totalHours)
# maximumDemand = np.random.rand(totalHours)

# dso = gameanalysis.StackelbergGame(DSO.numberOfAggregators)
# for i in range(DSO.numberOfAggregators):
#     agg = DSO.aggregators[i]
#     u = np.zeros((totalHours, totalHours))
#     for j in range(totalHours):
#         u[j, :] = dso.devfn(DSO.utilityFunction, agg, j)
#     dso.add_player(u)

# # Calculate the equilibrium strategy for the DSO
# dso_eq = dso.solve()[0]

# # Calculate the equilibrium strategies for each aggregator
# for i in range(DSO.numberOfAggregators):
#     agg = DSO.aggregators[i]
#     agg_strategies = []
#     for j in range(totalHours):
#         s = dso_eq.strategies[agg.index][j, :]
#         agg_strategies.append(s)
#     agg_eq = gameanalysis.regret.mixture_regret(
#         agg_strategies, DSO.aggregators[i].utilityFunction)
